# Crime Trail Blazers Data

In [2]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [3]:
# Name of the CSV file
file15 = 'Resources/2015_Portland_Crime_Data.csv'
file16 = 'Resources/2016_Portland_Crime_Data.csv'
file17 = 'Resources/2017_Portland_Crime_Data.csv'
file18 = 'Resources/2018_Portland_Crime_Data.csv'
file19 = 'Resources/2019YTD_Portland_Crime_Data.csv'
nba_data ='Resources/portland.games.stats.csv'

In [4]:
# The correct encoding must be used to read the CSV in pandas
df_2015 = pd.read_csv(file15, encoding="ISO-8859-1")
df_2016 = pd.read_csv(file16, encoding="ISO-8859-1")
df_2017 = pd.read_csv(file17, encoding="ISO-8859-1")
df_2018 = pd.read_csv(file18, encoding="ISO-8859-1")
df_2019 = pd.read_csv(file18, encoding="ISO-8859-1")
nba_data = pd.read_csv(nba_data, encoding="ISO-8859-1")

## Portland Trailblazers Stats 2015-18

In [5]:
nba_data.head()

,Unnamed: 0,Team,Game,Date,Home,Opponent,WINorLOSS,TeamPoints,OpponentPoints,FieldGoals,...,Opp.FreeThrows,Opp.FreeThrowsAttempted,Opp.FreeThrows.,Opp.OffRebounds,Opp.TotalRebounds,Opp.Assists,Opp.Steals,Opp.Blocks,Opp.Turnovers,Opp.TotalFouls
0,1108,POR,1,10/29/2014,Home,OKC,W,106,89,39,...,21,26,0.808,14,43,19,9,5,13,28
1,2108,POR,2,10/31/2014,Away,SAC,L,94,103,34,...,31,35,0.886,14,49,14,6,2,10,24
2,3108,POR,3,11/2/2014,Home,GSW,L,90,95,37,...,17,21,0.810,7,41,22,11,7,17,16
3,4108,POR,4,11/4/2014,Home,CLE,W,101,82,37,...,11,14,0.786,13,41,18,5,3,14,19
4,5108,POR,5,11/6/2014,Home,DAL,W,108,87,40,...,16,16,1.000,8,34,14,6,4,7,17


In [6]:
# add Day of Week
nba_data['DOW']= pd.to_datetime(nba_data['Date']).dt.dayofweek

In [7]:
# Change date format and reduce dates to 2015 - 2018
nba_data['Date'] = pd.to_datetime(nba_data['Date'])
start_date = '01/01/2015'
end_date = '12/31/2018'
mask = (nba_data['Date'] >= start_date) & (nba_data['Date'] <= end_date)
nba_data = nba_data.loc[mask]

In [58]:
# Created a new dataframe using the data_df to display these columns:
new_nba_df = nba_data[['Date','Team','Game','Home','WINorLOSS', 'DOW']]

In [9]:
# Export the Trail Blazers Data into a csv
new_nba_df.to_csv('Output/trail_blazers.csv')

## Crime Data

In [10]:
# Concatinate all crime data
concat_crime_df = pd.concat([df_2015, df_2016, df_2017, df_2018, df_2019])

In [11]:
# add Day of Week
concat_crime_df['DOW']= pd.to_datetime(concat_crime_df['OccurDate']).dt.dayofweek

In [12]:
# Created a new dataframe using the data_df to display these columns:
new_crime_df = concat_crime_df[['OccurDate','Neighborhood','OffenseCategory','DOW']]

In [59]:
# Rename date field
new_crime_df = new_crime_df.rename(columns={"OccurDate":"Date"})

In [14]:
# Filter out reports of crimes prior to 2015
new_crime_df['Date'] = pd.to_datetime(new_crime_df['Date'])
start_date = '01/01/2015'
end_date = '12/31/2018'
mask = (new_crime_df['Date'] >= start_date) & (new_crime_df['Date'] <= end_date)
new_crime_df = new_crime_df.loc[mask]
new_crime_df

,Date,Neighborhood,OffenseCategory,DOW
0,2015-05-12,Piedmont,Assault Offenses,1
1,2015-05-01,Buckman West,Assault Offenses,4
2,2015-05-01,University Park,Assault Offenses,4
3,2015-05-01,Madison South,Assault Offenses,4
4,2015-05-01,Madison South,Kidnapping/Abduction,4
...,...,...,...,...
60383,2018-05-07,NaN,Drug/Narcotic Offenses,0
60384,2018-07-25,NaN,Assault Offenses,2
60385,2018-10-02,Arlington Heights,Vandalism,1
60386,2018-02-27,NaN,Larceny Offenses,1


In [15]:
# Create new dataframe specific to Date occurances
df_dow = new_crime_df[["Date", "DOW"]]

In [16]:
# Run for loop to count number of crimes for each date
unique_dates = list(df_dow["Date"].unique())

listy = []
for i in unique_dates:
    holdy = df_dow.loc[df_dow["Date"] == i].reset_index(drop=True)
    holdy1 = holdy["Date"].count()
    holder_dictionary = {"Date": i,
                         "Num_times": holdy1,
                         "DOW": holdy["DOW"][0]}
    listy.append(holder_dictionary)

In [17]:
# Create datafrom of the daily crime count
crime_count_df = pd.DataFrame(listy)
crime_count_df

,Date,Num_times,DOW
0,2015-05-12,161,1
1,2015-05-01,196,4
2,2015-05-02,176,5
3,2015-05-03,130,6
4,2015-05-04,143,0
...,...,...,...
1454,2018-12-28,276,4
1455,2018-12-29,226,5
1456,2018-12-30,242,6
1457,2018-12-31,176,0


In [18]:
# Export the crime count into a csv
crime_count_df.to_csv('Output/total_daily_crime_count.csv')

## Trail Blazers and Crime Data

In [63]:
# Merge Trail Blazer and Crime Data
merge_table = pd.merge(new_crime_df, new_nba_df, on="Date", how="left")

In [33]:
# Modify merged table to add columns for further analysis
# Add Year, Month and Month_Year data
merge_table["HasGame"] = merge_table['Game'] > 0
merge_table["GameStatus"] = merge_table["Home"] + merge_table["WINorLOSS"] 
merge_table["GameStatus"] = merge_table["GameStatus"].fillna('NoGame') 
merge_table["Year"] = pd.DatetimeIndex(merge_table["Date"]).year
merge_table["Month"] = pd.DatetimeIndex(merge_table["Date"]).month
merge_table["Month_Year"] = pd.to_datetime(merge_table["Date"]).dt.to_period('M')

In [40]:
# Created a new dataframe to display these columns:
month_crime_df = merge_table[['Date','Year','Month','Month_Year']]
month_crime_df.head()

,Date,Year,Month,Month_Year
0,2015-05-12,2015,5,2015-05
1,2015-05-01,2015,5,2015-05
2,2015-05-01,2015,5,2015-05
3,2015-05-01,2015,5,2015-05
4,2015-05-01,2015,5,2015-05


In [39]:
# Export the crime count into a csv
month_crime_df.to_csv('Output/month_year_crime.csv')

In [23]:
grouped_table = merge_table.groupby(['DOW_x', 'GameStatus']).agg(
    {'OffenseCategory':'count',
     'Date':'nunique'
    }
)
grouped_table['AverageDailyCrimes'] = grouped_table['OffenseCategory']/grouped_table['Date']
grouped_table

OffenseCategory  Date  AverageDailyCrimes
DOW_x GameStatus                                           
0     AwayL                  2774    17          163.176471
      AwayW                  1723     8          215.375000
      HomeL                   571     4          142.750000
      HomeW                   992     8          124.000000
      NoGame                33073   172          192.284884
1     AwayL                  1825     8          228.125000
      AwayW                  1790     8          223.750000
      HomeL                  1591     9          176.777778
      HomeW                  2567    14          183.357143
      NoGame                31307   169          185.248521
2     AwayL                  2857    19          150.368421
      AwayW                   715     6          119.166667
      HomeL                  1035     8          129.375000
      HomeW                  3120    20          156.000000
      NoGame                31926   155          205.974194
3     AwayL                  1183     8          147.875000
      AwayW                   187     1          187.000000
      HomeL                  1026     7          146.571429
      HomeW                  3182    18          176.777778
      NoGame                33815   175          193.228571
4     AwayL                  2230    15          148.666667
      AwayW                  2305    12          192.083333
      HomeL                  1372     7          196.000000
      HomeW                  2040    11          185.454545
      NoGame                33812   164          206.170732
5     AwayL                  1167    10          116.700000
      AwayW                  1851    10          185.100000
      HomeL                  1081     8          135.125000
      HomeW                  3180    20          159.000000
      NoGame                31932   160          199.575000
6     AwayL                  1565     9          173.888889
      AwayW                  2054    16          128.375000
      HomeL                   826     6          137.666667
      HomeW                  1409     7          201.285714
      NoGame                30215   170          177.735294

In [24]:
# Export Day of Week Crime table
grouped_table.to_csv('Output/day_of_week_table.csv')

In [62]:
total_table = merge_table.groupby(['GameStatus']).agg(
    {'OffenseCategory':'count',
     'Date':'nunique'
    }
)
total_table['AverageCrime'] = total_table['OffenseCategory']/total_table['Date']

In [26]:
# Export Overal Crime table
total_table.to_csv('Output/game_status_table.csv')

In [61]:
# Merge Trail Blazer and Crime Count
count_table = pd.merge(crime_count_df, new_nba_df, on="Date", how="left")
count_table["HasGame"] = count_table['Game'] > 0
count_table["Month_Year"] = pd.to_datetime(count_table["Date"]).dt.to_period('M')
count_table = count_table[['Date','Num_times','HasGame','Month_Year']]

In [57]:
count_table.to_csv('Output/gameday_table.csv')